In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-11-14 14:51:04--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M  --.-KB/s    in 0.1s    

2024-11-14 14:51:04 (169 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
user_rating_counts = df_ratings['user'].value_counts()
valid_users = user_rating_counts[user_rating_counts >= 200].index
filtered_ratings = df_ratings[df_ratings['user'].isin(valid_users)]


book_rating_counts = filtered_ratings['isbn'].value_counts()
valid_books = book_rating_counts[book_rating_counts >= 100].index
filtered_ratings = filtered_ratings[filtered_ratings['isbn'].isin(valid_books)]

user_item_matrix = filtered_ratings.pivot_table(index='user', columns='isbn', values='rating', fill_value=0)

model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=3)
model.fit(user_item_matrix.T)


In [ ]:
def get_recommends(book = ""):

    book_isbn = df_books[df_books['title'] == book]['isbn'].values[0]

    book_index = user_item_matrix.columns.get_loc(book_isbn)


    book_ratings = user_item_matrix.iloc[:, book_index].values.reshape(1, -1)
    distances, indices = model.kneighbors(book_ratings, n_neighbors=6)


    recommendations = []
    for i in range(1, len(indices[0])):
        recommended_isbn = user_item_matrix.columns[indices[0][i]]
        recommended_title = df_books[df_books['isbn'] == recommended_isbn]['title'].values[0]
        distance = distances[0][i]
        recommendations.append([recommended_title, distance])

    return [book, recommendations]

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()